In [ ]:
import sys

from synthcity.plugins import Plugins
from synthcity.benchmark import Benchmarks
import synthcity.logger as log

from pathlib import Path
from datasets import get_dataset

log.add(sink=sys.stderr, level="INFO")

plugins = Plugins().list()

out_dir = Path("output")

gain_scenarios = [
    (
        "uncensoring_with_date",
        {
            "uncensoring_model": "date",
            "tte_strategy": "uncensoring",
            "dataloader_sampling_strategy": "none",
        },
    ),
    (
        "survival_function_with_date",
        {
            "uncensoring_model": "date",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "none",
        },
    ),
    (
        "survival_function_regression_no_sampling",
        {
            "uncensoring_model": "survival_function_regression",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "none",
        },
    ),
    (
        "survival_function_regression_imbalanced_cens_sampling",
        {
            "uncensoring_model": "survival_function_regression",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "imbalanced_censoring",
        },
    ),
    (
        "survival_function_regression_imbalanced_time_cens_sampling",
        {
            "uncensoring_model": "survival_function_regression",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "imbalanced_time_censoring",
        },
    ),
]


def evaluate_dataset(dataset: str, scenarios: list):
    df, duration_col, event_col, time_horizons = get_dataset(dataset)

    for scenario_name, scenario_args in scenarios:
        bkp = out_dir / f"sources_of_gain_{dataset}_{scenario_name}.bkp"

        print(scenario_name, flush=True)
        Plugins().get("survival_gan", **scenario_args)
        continue

        if bkp.exists():
            score = load_from_file(bkp)
        else:
            score = Benchmarks.evaluate(
                ["survival_gan"],
                df,
                task_type="survival_analysis",
                target_column=event_col,
                time_to_event_column=duration_col,
                time_horizons=time_horizons,
                synthetic_size=len(df),
                repeats=repeats,
                **scenario_args,
            )
            save_to_file(bkp, score)

        print("Scenario", scenario_name, scenario_args)
        Benchmarks.print(score)

    return
    return score

In [ ]:
repeats = 3

## AIDS dataset

In [ ]:
base_score = evaluate_dataset("aids", gain_scenarios)

## Metabric

In [ ]:
base_score = evaluate_dataset("metabric", gain_scenarios)

## CUTRACT

In [ ]:
base_score = evaluate_dataset("cutract", gain_scenarios)

## MAGGIC dataset 

In [ ]:
base_score = evaluate_dataset("maggic", gain_scenarios)

## SEER prostate 

In [ ]:
base_score = evaluate_dataset("seer", gain_scenarios)